In [9]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDRegressor
from tqdm import tqdm

In [10]:
df_final = pd.read_excel("data/processed/df_final_norm.xlsx")
df_final = df_final.sort_values("Date").reset_index(drop=True) 

In [ ]:
#FONCTIONS 

def generate_time_splits(df, date_col='Date', train_ratio=0.85, step_months=12, val_months=12, test_months=12):
    """
    Génère des splits temporels rolling window : train / val / test

    Paramètres :
    - df : df trié par date
    - date_col : nom de la colonne des dates
    - train_ratio : part du dataset à allouer au train initial (ex: 0.85)
    - step_months : taille du pas de glissement (ex: 12)
    - val_months : taille de la validation (ex: 12)
    - test_months : taille du test (ex: 12)

    Retour :
    - liste de tuples (train_idx, val_idx, test_idx)
    """
    df = df.sort_values(date_col).reset_index(drop=True)
    dates = sorted(df[date_col].unique())
    total_months = len(dates)
    train_initial = int(total_months * train_ratio)

    splits = []

    for i in range(0, total_months - train_initial - val_months - test_months + 1, step_months):
        train_end = train_initial + i
        val_start = train_end
        val_end = val_start + val_months
        test_start = val_end
        test_end = test_start + test_months

        if test_end > total_months:
            break

        train_dates = dates[:train_end]
        val_dates = dates[val_start:val_end]
        test_dates = dates[test_start:test_end]

        train_idx = df[df[date_col].isin(train_dates)].index.tolist()
        val_idx = df[df[date_col].isin(val_dates)].index.tolist()
        test_idx = df[df[date_col].isin(test_dates)].index.tolist()

        splits.append((train_idx, val_idx, test_idx))
    return splits 


#exclude cols
exclude_cols = ["Ticker", "Date", "excess_return"]
def get_X_y(df, idx, target="excess_return", exclude_cols=None):
    if exclude_cols is None:
        exclude_cols = ["Ticker", "Date", target]
    subset = df.loc[idx]
    x = subset.drop(columns=exclude_cols)
    y = subset[target]
    return x, y

#PLS 
def pls_tuning(X_train, y_train, X_val, y_val, X_test, y_test, max_components=28):
    """
    Tuning et estimation du modèle PLS selon Gu et al. (2020)
    """
    candidate_ks = np.arange(1, min(max_components, X_train.shape[1]) + 1) #test de 1 à 28 composants 
    mse_val_grid = []

    for k in candidate_ks:

        #entraîne sur train 
        model = PLSRegression(n_components=k, scale=False)
        model.fit(X_train, y_train)
        preds_val = model.predict(X_val).ravel()
        mse = mean_squared_error(y_val, preds_val)
        mse_val_grid.append(mse)
    #meilleur k : minimise MSE 
    best_k = candidate_ks[np.argmin(mse_val_grid)]

    # Réentraînement sur train + val
    X_trainval = pd.concat([X_train, X_val])
    y_trainval = pd.concat([y_train, y_val])
    model_final = PLSRegression(n_components=best_k, scale=False)
    model_final.fit(X_trainval, y_trainval)

    preds_test = model_final.predict(X_test).ravel()
    r2_test = 1 - np.sum((y_test - preds_test)**2) / np.sum(y_test**2)

    return preds_test, best_k, mse_val_grid, r2_test

#PCR

def pcr_tuning(X_train, y_train, X_val, y_val, X_test, y_test, max_components=28):
    """
    Tuning et estimation du modèle PCR (Principal Component Regression)
    """
    candidate_ks = np.arange(1, min(max_components, X_train.shape[1]) + 1)
    mse_val_grid = []

    for k in candidate_ks:
        pca = PCA(n_components=k)
        pca.fit(X_train)
        X_train_pca = pca.transform(X_train)
        X_val_pca = pca.transform(X_val)

        model = LinearRegression()
        model.fit(X_train_pca, y_train)
        preds_val = model.predict(X_val_pca).ravel()
        mse = mean_squared_error(y_val, preds_val)
        mse_val_grid.append(mse)

    # Meilleur k : minimise le MSE
    best_k = candidate_ks[np.argmin(mse_val_grid)]

    # Réentraînement sur train + val
    X_trainval = pd.concat([X_train, X_val])
    y_trainval = pd.concat([y_train, y_val])
    pca_final = PCA(n_components=best_k)
    X_trainval_pca = pca_final.fit_transform(X_trainval)
    model_final = LinearRegression()
    model_final.fit(X_trainval_pca, y_trainval)

    # Prédictions sur test
    X_test_pca = pca_final.transform(X_test)
    preds_test = model_final.predict(X_test_pca).ravel()
    r2_test = 1 - np.sum((y_test - preds_test)**2) / np.sum(y_test**2)

    return preds_test, best_k, mse_val_grid, r2_test

def enet_tuning(X_train, y_train, X_val, y_val, X_test, y_test,
                      alpha_grid=None, l1_ratio=0.5):
    """
    Tuning et estimation du modèle ElasticNet (Huber loss) sans standardisation,
    en supposant que les données ont déjà été normalisées entre -1 et 1.
    """
    if alpha_grid is None:
        alpha_grid = np.logspace(10E-5, -10E-1, num=20)

    mse_val_grid = []

    for alpha in alpha_grid:
        model = SGDRegressor(loss='huber',
                             penalty='elasticnet',
                             alpha=alpha,
                             l1_ratio=l1_ratio,
                             max_iter=1000,
                             tol=1e-3,
                             random_state=42)
        model.fit(X_train, y_train)
        preds_val = model.predict(X_val)
        mse = mean_squared_error(y_val, preds_val)
        mse_val_grid.append(mse)

    best_alpha = alpha_grid[np.argmin(mse_val_grid)]

    # Réentraînement sur train + val
    X_trainval = pd.concat([X_train, X_val])
    y_trainval = pd.concat([y_train, y_val])
    model_final = SGDRegressor(loss='huber',
                               penalty='elasticnet',
                               alpha=best_alpha,
                               l1_ratio=l1_ratio,
                               max_iter=1000,
                               tol=1e-3,
                               random_state=42)
    model_final.fit(X_trainval, y_trainval)

    # Prédictions sur test
    preds_test = model_final.predict(X_test)
    r2_test = 1 - np.sum((y_test - preds_test)**2) / np.sum(y_test**2)

    return preds_test, best_alpha, mse_val_grid, r2_test

In [12]:
#SPLITS : 85% de la data, rolling tous les 12 ans

splits = generate_time_splits(df_final, date_col="Date", train_ratio=0.85, step_months=12, val_months=12, test_months=12)


# Résultats PLS
y_preds_pls = []
y_test_pls = []
dates_pls = []
r2_by_date = []
best_k_by_date = []

# Résultats OLS
y_preds_ols = []
y_test_ols = []
dates_ols = []
r2_by_date_ols = []

# Résultats PCR
y_preds_pcr = []
y_test_pcr = []
dates_pcr = []
r2_by_date_pcr = []
best_k_by_date_pcr = []

#Elastic Net
y_preds_enh = []
y_test_enh = []
dates_enh = []
r2_by_date_enh = []
best_alpha_by_date_enh = []

In [13]:
#Linear models 

for train_idx, val_idx, test_idx in tqdm(splits):
    X_train, y_train = get_X_y(df_final, train_idx)
    X_val, y_val = get_X_y(df_final, val_idx)
    X_test, y_test = get_X_y(df_final, test_idx)

    #OLS
    X_trainval = pd.concat([X_train, X_val])
    y_trainval = pd.concat([y_train, y_val])
    model_ols = LinearRegression()
    model_ols.fit(X_trainval, y_trainval)
    preds_test_ols = model_ols.predict(X_test)
    r2_test_ols = 1 - np.sum((y_test - preds_test_ols)**2) / np.sum(y_test**2) if np.sum(y_test**2) > 0 else np.nan
    y_preds_ols.append(preds_test_ols)
    y_test_ols.append(y_test.values)
    dates_ols.append(df_final.loc[test_idx, "Date"].values)
    r2_by_date_ols.append(r2_test_ols)


    #PLS
    preds_test, best_k, mse_grid, r2_test = pls_tuning(X_train, y_train, X_val, y_val, X_test, y_test)
    y_preds_pls.append(preds_test)
    y_test_pls.append(y_test.values)
    dates_pls.append(df_final.loc[test_idx, "Date"].values)
    r2_by_date.append(r2_test)
    best_k_by_date.append(best_k)


    # PCR
    preds_test_pcr, best_k_pcr, _, r2_test_pcr = pcr_tuning(X_train, y_train, X_val, y_val, X_test, y_test)
    y_preds_pcr.append(preds_test_pcr)
    y_test_pcr.append(y_test.values)
    dates_pcr.append(df_final.loc[test_idx, "Date"].values)
    r2_by_date_pcr.append(r2_test_pcr)
    best_k_by_date_pcr.append(best_k_pcr)

    #Elastic Net
    preds_test_enh, best_alpha_enh, _, r2_test_enh = enet_tuning(X_train, y_train, X_val, y_val, X_test, y_test)
    y_preds_enh.append(preds_test_enh)
    y_test_enh.append(y_test.values)
    dates_enh.append(df_final.loc[test_idx, "Date"].values)
    r2_by_date_enh.append(r2_test_enh)
    best_alpha_by_date_enh.append(best_alpha_enh)


 
# --- R² OOS global PLS
y_preds_all = np.concatenate(y_preds_pls)
y_test_all = np.concatenate(y_test_pls)
dates_all = np.concatenate(dates_pls)
R2OOS_PLS = 1 - np.sum((y_test_all - y_preds_all) ** 2) / np.sum(y_test_all ** 2)
print(f"R² OOS global PLS : {R2OOS_PLS:.4f}")

# --- R² OOS global OLS
y_preds_ols_all = np.concatenate(y_preds_ols)
y_test_ols_all = np.concatenate(y_test_ols)
R2OOS_OLS = 1 - np.sum((y_test_ols_all - y_preds_ols_all) ** 2) / np.sum(y_test_ols_all ** 2)
print(f"R² OOS global OLS : {R2OOS_OLS:.4f}")

# --- R² OOS global PCR
y_preds_all_pcr = np.concatenate(y_preds_pcr)
y_test_all_pcr = np.concatenate(y_test_pcr)
R2OOS_PCR = 1 - np.sum((y_test_all_pcr - y_preds_all_pcr) ** 2) / np.sum(y_test_all_pcr ** 2)
print(f"R² OOS global PCR : {R2OOS_PCR:.4f}")

#Enet 
y_preds_enh_all = np.concatenate(y_preds_enh)
y_test_enh_all = np.concatenate(y_test_enh)
dates_enh_all = np.concatenate(dates_enh)
R2OOS_ENH = 1 - np.sum((y_test_enh_all - y_preds_enh_all) ** 2) / np.sum(y_test_enh_all ** 2)
print(f"R² OOS global ENH : {R2OOS_ENH:.4f}")
print(f"Best alpha (ENH) pour la fenêtre : {best_alpha_enh:.6f}")

# --- Résultats détaillés tout
df_r2 = pd.DataFrame({
    "date": [d[0] for d in dates_pls],
    "R2_test_pls": r2_by_date,
    "Best_k_pls": best_k_by_date,
    "R2_test_ols": r2_by_date_ols,
    "R2_test_pcr": r2_by_date_pcr,
    "Best_k_pcr": best_k_by_date_pcr,
    "best_alpha_by_date_enh" : best_alpha_by_date_enh
})

df_preds = pd.DataFrame({
    "date": np.concatenate(dates_pls),
    "y_test": y_test_all,
    "y_pred_ols": y_preds_ols_all,
    "y_pred_pls": y_preds_all,
    "y_pred_pcr": y_preds_all_pcr,
    "y_pred_enh": y_preds_enh_all
})


100%|██████████| 3/3 [00:32<00:00, 10.91s/it]

R² OOS global PLS : 0.0253
R² OOS global OLS : 0.0263
R² OOS global PCR : 0.0237
R² OOS global ENH : -0.1376
Best alpha (ENH) pour la fenêtre : 0.886060


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def analyse_modele(y_test_all, y_preds_all, nom_modele):
    erreur = y_preds_all - y_test_all
    print(f"\n--- Analyse {nom_modele} ---")
    print(f"Moyenne y_test : {np.mean(y_test_all):.6f}")
    print(f"Moyenne prédictions : {np.mean(y_preds_all):.6f}")
    print(f"MSE : {mean_squared_error(y_test_all, y_preds_all):.6f}")
    print(f"MAE : {mean_absolute_error(y_test_all, y_preds_all):.6f}")
    print(f"Écart-type des erreurs : {np.std(erreur):.6f}")
    print(f"Erreur moyenne : {np.mean(erreur):.6f}")
    print(f"Min préd : {np.min(y_preds_all):.6f} | Max préd : {np.max(y_preds_all):.6f}")
    print(f"Min vrai : {np.min(y_test_all):.6f} | Max vrai : {np.max(y_test_all):.6f}")

analyse_modele(y_test_all, y_preds_all, "PLS")
analyse_modele(y_test_ols_all, y_preds_ols_all, "OLS")
analyse_modele(y_test_all_pcr, y_preds_all_pcr, "PCR")
analyse_modele(y_test_enh_all, y_preds_enh_all, "ElasticNet")



--- Analyse PLS ---
Moyenne y_test : 0.008396
Moyenne prédictions : 0.012086
MSE : 0.006309
MAE : 0.058339
Écart-type des erreurs : 0.079342
Erreur moyenne : 0.003689
Min préd : -0.029126 | Max préd : 0.070661
Min vrai : -0.592751 | Max vrai : 0.532851

--- Analyse OLS ---
Moyenne y_test : 0.008396
Moyenne prédictions : 0.012073
MSE : 0.006302
MAE : 0.058370
Écart-type des erreurs : 0.079301
Erreur moyenne : 0.003677
Min préd : -0.036832 | Max préd : 0.070653
Min vrai : -0.592751 | Max vrai : 0.532851

--- Analyse PCR ---
Moyenne y_test : 0.008396
Moyenne prédictions : 0.012082
MSE : 0.006319
MAE : 0.058399
Écart-type des erreurs : 0.079410
Erreur moyenne : 0.003686
Min préd : -0.029209 | Max préd : 0.070653
Min vrai : -0.592751 | Max vrai : 0.532851

--- Analyse ElasticNet ---
Moyenne y_test : 0.008396
Moyenne prédictions : 0.038939
MSE : 0.007363
MAE : 0.064016
Écart-type des erreurs : 0.080189
Erreur moyenne : 0.030543
Min préd : 0.033017 | Max préd : 0.047358
Min vrai : -0.592751 

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# --- Fonction de tuning RF
def rf_tuning(X_train, y_train, X_val, y_val, X_test, y_test,
              n_estimators_list=[100, 300],
              max_depth_list=[10, 20, None],
              min_samples_leaf_list=[1, 5],
              max_features_list=['sqrt', 'log2']):

    best_mse = np.inf
    best_params = None

    for n in n_estimators_list:
        for d in max_depth_list:
            for leaf in min_samples_leaf_list:
                for feat in max_features_list:
                    model = RandomForestRegressor(
                        n_estimators=n,
                        max_depth=d,
                        min_samples_leaf=leaf,
                        max_features=feat,
                        random_state=42,
                        n_jobs=-1
                    )
                    model.fit(X_train, y_train)
                    preds_val = model.predict(X_val)
                    mse = mean_squared_error(y_val, preds_val)

                    if mse < best_mse:
                        best_mse = mse
                        best_params = (n, d, leaf, feat)

    # Réentraînement sur train + val
    X_trainval = pd.concat([X_train, X_val])
    y_trainval = pd.concat([y_train, y_val])
    model_final = RandomForestRegressor(
        n_estimators=best_params[0],
        max_depth=best_params[1],
        min_samples_leaf=best_params[2],
        max_features=best_params[3],
        random_state=42,
        n_jobs=-1
    )
    model_final.fit(X_trainval, y_trainval)
    preds_test = model_final.predict(X_test)
    r2_test = 1 - np.sum((y_test - preds_test)**2) / np.sum(y_test**2) if np.sum(y_test**2) > 0 else np.nan

    return preds_test, best_params, r2_test


y_preds_rf = []
y_test_rf = []
dates_rf = []
r2_by_date_rf = []
best_params_by_date_rf = []

# --- Génération des splits
splits = generate_time_splits(df_final, date_col="Date", train_ratio=0.85, step_months=12, val_months=12, test_months=12)

# --- Boucle principale RF
for train_idx, val_idx, test_idx in tqdm(splits):
    X_train, y_train = get_X_y(df_final, train_idx)
    X_val, y_val = get_X_y(df_final, val_idx)
    X_test, y_test = get_X_y(df_final, test_idx)

    preds_test_rf, best_params_rf, r2_test_rf = rf_tuning(X_train, y_train, X_val, y_val, X_test, y_test)

    y_preds_rf.append(preds_test_rf)
    y_test_rf.append(y_test.values)
    dates_rf.append(df_final.loc[test_idx, "Date"].values)
    r2_by_date_rf.append(r2_test_rf)
    best_params_by_date_rf.append(best_params_rf)

# --- R² OOS global RF
y_preds_rf_all = np.concatenate(y_preds_rf)
y_test_rf_all = np.concatenate(y_test_rf)
R2OOS_RF = 1 - np.sum((y_test_rf_all - y_preds_rf_all) ** 2) / np.sum(y_test_rf_all ** 2)
print(f"R² OOS global RF : {R2OOS_RF:.4f}")

# --- Résultats dans df
df_r2_rf = pd.DataFrame({
    "date": [d[0] for d in dates_rf],
    "R2_test_rf": r2_by_date_rf,
    "best_params_rf": best_params_by_date_rf
})

df_preds_rf = pd.DataFrame({
    "date": np.concatenate(dates_rf),
    "y_test": y_test_rf_all,
    "y_pred_rf": y_preds_rf_all
})


100%|██████████| 3/3 [03:35<00:00, 71.80s/it]

R² OOS global RF : 0.0321


In [16]:
analyse_modele(y_test_rf_all, y_preds_rf_all, "Random Forest")


--- Analyse Random Forest ---
Moyenne y_test : 0.008396
Moyenne prédictions : 0.010733
MSE : 0.006265
MAE : 0.058155
Écart-type des erreurs : 0.079117
Erreur moyenne : 0.002337
Min préd : -0.041196 | Max préd : 0.048767
Min vrai : -0.592751 | Max vrai : 0.532851


In [17]:
r2_in_ols = []
r2_in_pls = []
r2_in_pcr = []
r2_in_enh = []


In [ ]:
    # concaténation train + val
    X_trainval = pd.concat([X_train, X_val])
    y_trainval = pd.concat([y_train, y_val])

    # R² in-sample naïf OLS
    y_in_ols = model_ols.predict(X_trainval)
    r2_in_ols.append(1 - np.mean((y_trainval - y_in_ols) ** 2) / np.var(y_trainval))

    # R² in-sample naïf PLS
    pls_model_in = PLSRegression(n_components=best_k, scale=False)
    pls_model_in.fit(X_trainval, y_trainval)
    y_in_pls = pls_model_in.predict(X_trainval).ravel()
    r2_in_pls.append(1 - np.mean((y_trainval - y_in_pls) ** 2) / np.var(y_trainval))

    # R² in-sample naïf PCR
    pca_in = PCA(n_components=best_k_pcr)
    X_trainval_pca = pca_in.fit_transform(X_trainval)
    model_pcr_in = LinearRegression().fit(X_trainval_pca, y_trainval)
    y_in_pcr = model_pcr_in.predict(X_trainval_pca)
    r2_in_pcr.append(1 - np.mean((y_trainval - y_in_pcr) ** 2) / np.var(y_trainval))

    # R² in-sample naïf ENH
    model_enh_in = SGDRegressor(loss='huber',
                                penalty='elasticnet',
                                alpha=best_alpha_enh,
                                l1_ratio=0.5,
                                max_iter=1000,
                                tol=1e-3,
                                random_state=42)
    model_enh_in.fit(X_trainval, y_trainval)
    y_in_enh = model_enh_in.predict(X_trainval)
    r2_in_enh.append(1 - np.mean((y_trainval - y_in_enh) ** 2) / np.var(y_trainval))  

In [ ]:
print(f"R² in-sample OLS (naïf): {np.mean(r2_in_ols):.4f}")
print(f"R² in-sample PLS (naïf): {np.mean(r2_in_pls):.4f}")
print(f"R² in-sample PCR (naïf): {np.mean(r2_in_pcr):.4f}")
print(f"R² in-sample ENH (naïf): {np.mean(r2_in_enh):.4f}")

R² in-sample OLS (naïf): 0.0206
R² in-sample PLS (naïf): 0.0206
R² in-sample PCR (naïf): 0.0206
R² in-sample ENH (naïf): -0.0730


In [23]:
# Calcul de la matrice de corrélation
corr_matrix = X_trainval.corr()

# Affichage rapide : top 10 paires les plus corrélées (hors diagonale)
corr_pairs = corr_matrix.unstack()
corr_pairs = corr_pairs[corr_pairs.index.get_level_values(0) != corr_pairs.index.get_level_values(1)]
corr_pairs = corr_pairs.abs().sort_values(ascending=False)

print("variables les plus corrélées :\n")
print(corr_pairs.head(45))


variables les plus corrélées :

beta          beta_squared    1.000000
beta_squared  beta            1.000000
illiq         dolvol_lag2     0.907136
dolvol_lag2   illiq           0.907136
illiq         mvel1           0.884901
mvel1         illiq           0.884901
stdturn       turn            0.814659
turn          stdturn         0.814659
mvel1         dolvol_lag2     0.807527
dolvol_lag2   mvel1           0.807527
retvol        maxret          0.803178
maxret        retvol          0.803178
invest        chinv           0.654425
chinv         invest          0.654425
idiovol       turn            0.648615
turn          idiovol         0.648615
retvol        idiovol         0.626190
idiovol       retvol          0.626190
retvol        turn            0.610341
turn          retvol          0.610341
beta          retvol          0.609475
retvol        beta_squared    0.609475
beta_squared  retvol          0.609475
retvol        beta            0.609475
chmom         mom6m           0.

In [30]:
r2_in_rf = []  # liste pour stocker les R² in-sample par split

for train_idx, val_idx, test_idx in tqdm(splits):
    X_train, y_train = get_X_y(df_final, train_idx)
    X_val, y_val = get_X_y(df_final, val_idx)
    X_test, y_test = get_X_y(df_final, test_idx)

    # tuning sur train/val/test
    preds_test_rf, best_params_rf, r2_test_rf = rf_tuning(X_train, y_train, X_val, y_val, X_test, y_test)

    # --- R² in-sample naïf RF (sur train+val)
    X_trainval = pd.concat([X_train, X_val])
    y_trainval = pd.concat([y_train, y_val])
    model_rf_in = RandomForestRegressor(
        n_estimators=best_params_rf[0],
        max_depth=best_params_rf[1],
        min_samples_leaf=best_params_rf[2],
        max_features=best_params_rf[3],
        random_state=42,
        n_jobs=-1
    )
    model_rf_in.fit(X_trainval, y_trainval)
    y_in_rf = model_rf_in.predict(X_trainval)
    r2_val = 1 - np.mean((y_trainval - y_in_rf) ** 2) / np.var(y_trainval)
    r2_in_rf.append(r2_val)

    # --- (le reste de ton stockage pour OOS)
    y_preds_rf.append(preds_test_rf)
    y_test_rf.append(y_test.values)
    dates_rf.append(df_final.loc[test_idx, "Date"].values)
    r2_by_date_rf.append(r2_test_rf)
    best_params_by_date_rf.append(best_params_rf)

# après la boucle :
print(f"R² in-sample moyen RF : {np.mean(r2_in_rf):.4f}")


100%|██████████| 3/3 [03:53<00:00, 77.96s/it]

R² in-sample moyen RF : 0.2808
